In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tabulate as tabulate
import matplotlib.colors as mcolors
from IPython.core.display import HTML
from tabulate import tabulate
from bs4 import BeautifulSoup
import requests
import re

## hier kommt jetzt nochmal eine neue Analyse von table bzw der relativen frequency besonders bzgl der monate, in R erkenne ich nicht 


In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

hat funktioniert, zwar nimmt das alte programm die hälfte statt 1982 / 1983 cut zu setzen aber ich habe das jetzt besser gemacht.

# generelle Häufigkeit ERA5 vs CMIP6 models- simulated patterns/ proj patterns

In [4]:
###Definitionssachen
#WICHTIG: für sim pattern ist die reihenfolge so wie in assignment und hier im programm
#WICHTIG:  für proj pattern ist die reihenfolge die reihenfolge von era5 historical

#models =["ACCESS-CM2", "ACCESS-ESM1", "CESM2-WACCM", "CNRM-CM6-1", "CNRM-CM6-1-HR", "CNRM-ESM2-1", 
 #        "GFDL-CM4", "GFDL-ESM4","HadGEM3-GC31-LL", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR", "MIROC6", 
  #       "MPI-ESM1-2-HR", "MPI-ESM1-2-LR", "MRI-ESM2-0", "NorESM2-MM", "UKESM1-0-LL"]
models=["KACE-1-0-G", "MIROC-ES2L", "NESM3", "NorESM2-LM"]
aufgabe = "Projected patterns"
def styling_specific_cell1(x,row_idx,col_idx):
    color = style_df_cl1.iloc[row_idx, col_idx]
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] = color
    return df_styler
def styling_specific_cell2(x,row_idx,col_idx):
    color = style_df_cl2.iloc[row_idx, col_idx]
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] = color
    return df_styler
def styling_specific_cell3(x,row_idx,col_idx):
    color = style_df_cl3.iloc[row_idx, col_idx]
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] = color
    return df_styler
def styling_specific_cell4(x,row_idx,col_idx):
    color = style_df_cl4.iloc[row_idx, col_idx]
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] = color
    return df_styler
def styling_specific_cell5(x,row_idx,col_idx):
    color = style_df_cl5.iloc[row_idx, col_idx]
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] = color
    return df_styler
def significance_decider(val,signi, minmax):
        if signi<minmax[4]:
            return 'background-color: darkblue; color: white'
        elif signi>minmax[5]:
            return 'background-color: darkred; color: white'
        elif signi>minmax[4] and signi<minmax[2]:
            return 'background-color: blue; color: white'
        elif signi>minmax[2] and signi<minmax[0]:
            return 'background-color: skyblue; color: black'
        elif signi<minmax[5] and signi>minmax[3]:
            return 'background-color: red; color: black'
        elif signi<minmax[3] and signi>minmax[1]:
            return 'background-color: orange; color: black'
        else:
            return 'background-color: white; color: black'
seas="mjjaso"
months=6
months_name=[5,6,7,8,9,10]
month_len=[31,30,31,31,30,31]
months_official=["May", "June", "July", "August", "September", "October"]
days_per_year=sum(month_len)
years=30
days_per_period=days_per_year*years
zusatz="historical"
years_hist="1985-2014"
years_proj="1985-2014"
years_fut="2070-2099"
area="-90_90"
region="atl"
if region=="paz":
    cl1={}
    cl2={}
    cl3={}
    cl4={}
    cl5={}
else:
    scan={}
    atl={}
    naoplus={}
    naominus={}
    dipol={}

counts_hist = {}
counts_fut = {}
occurence_hist = {}
occurence_fut={}
difference_cl1={}
difference_cl2={}
difference_cl3={}
difference_cl4={}
difference_cl5={}

style_dict_cl1 = {}
for month in range(0,months):
    column_name=str(months_official[month])
    style_dict_cl1[column_name]= np.array([])
style_dict_cl1['all']=np.array([])

style_dict_cl2 = {}
for month in range(0,months):
    column_name=str(months_official[month])
    style_dict_cl2[column_name]= np.array([])
style_dict_cl2['all']=np.array([])

style_dict_cl3 = {}
for month in range(0,months):
    column_name=str(months_official[month])
    style_dict_cl3[column_name]= np.array([])
style_dict_cl3['all']=np.array([])

style_dict_cl4 = {}
for month in range(0,months):
    column_name=str(months_official[month])
    style_dict_cl4[column_name]= np.array([])
style_dict_cl4['all']=np.array([])

style_dict_cl5 = {}
for month in range(0,months):
    column_name=str(months_official[month])
    style_dict_cl5[column_name]= np.array([])
style_dict_cl5['all']=np.array([])


if aufgabe == "Simulated patterns":
    #model="ACCESS-CM2"
    for model in range(len(models)):
        path_2="N:/atm_glomod/user/jomuel001/CMIP6_models/"+models[model]+"/AREA."+area+"_89.7849_29.0866/CLUSTER/"
        path="N:/atm_glomod/user/jomuel001/CMIP6_models/"+models[model]+"/AREA."+area+"_89.7849_29.0866/CLUSTER/PLOTS/"
        datei_signi=r"KMEANSslp_hpa_"+models[model]+"_"+years_hist+".N_"+seas+"_atrbg_aacrm21_remapbnds_proj_on_EOF_ERA5"+years_proj+"slp_hpa_ERA5_"+years_proj+"_"+seas+"_atrbg_aacrm21_remapbnds_unnorm_5cluster10PC_high_low.month.signif_bootstrap.txt"
        datei_minmax=r"KMEANSslp_hpa_"+models[model]+"_"+years_hist+".N_"+seas+"_atrbg_aacrm21_remapbnds_proj_on_EOF_ERA5"+years_proj+"slp_hpa_ERA5_"+years_proj+"_"+seas+"_atrbg_aacrm21_remapbnds_unnorm_5cluster10PC_high_low.month.signif_bootstrap_minmaxwhiskers.txt"
        datei_assignment_hist=r"Assignment_"+region+"_historical"+years_hist+"_"+seas+".txt"
        datei_assignment_fut=r"Assignment_"+region+"_future"+years_fut+"_"+seas+".txt"
        assignment_hist=pd.read_csv("N:/atm_glomod/user/jomuel001/auswertung/TAYLOR/Cluster/"+datei_assignment_hist,delimiter="\t")
        assignment_fut=pd.read_csv("N:/atm_glomod/user/jomuel001/auswertung/TAYLOR/Cluster/"+datei_assignment_fut,delimiter="\t")
        #cluster werden zu era5 zugeordnet
        datei_occ_hist=r"KMEANSslp_hpa_"+models[model]+"_"+years_hist+".N_"+seas+"_atrbg_aacrm21_remapbnds_proj_on_EOF_ERA5"+years_proj+"slp_hpa_ERA5_"+years_proj+"_"+seas+"_atrbg_aacrm21_remapbnds_unnorm_5tabcluster_10tabPC_dates"
        datei_occ_fut=r"KMEANSslp_hpa_"+models[model]+"_"+years_fut+".N_"+seas+"_atrbg_aacHrm21_remapbnds_proj_on_EOF_ERA5"+years_proj+"slp_hpa_ERA5_"+years_proj+"_"+seas+"_atrbg_aacrm21_remapbnds_unnorm_5tabcluster_10tabPC_dates"

        significance=np.array(pd.read_csv(path+datei_signi, delimiter=' '))[~np.isnan(np.array(pd.read_csv(path+datei_signi, delimiter=' ')))].reshape(months,5)
        minmaxwhiskers=np.array(pd.read_csv(path+datei_minmax, delimiter=' '))[~np.isnan(np.array(pd.read_csv(path+datei_minmax, delimiter=' ')))].reshape(months,5,6)
        occurence_hist['all']=np.loadtxt(path_2+datei_occ_hist, delimiter=' ')
        occurence_fut['all']=np.loadtxt(path_2+datei_occ_fut, delimiter=' ')
        counts_hist['all']=np.bincount(np.round(occurence_hist['all'][:,3]).astype(int))/(days_per_period)
        counts_fut['all']=np.bincount(np.round(occurence_fut['all'][:,3]).astype(int))/(days_per_period)

        for month in range(0,months):
            occurence_hist[str(months_official[month])]=np.array([])
            occurence_fut[str(months_official[month])]=np.array([])
            for entry in range(len(occurence_hist['all'])):
                if occurence_hist['all'][entry,1]==months_name[month]:
                    occurence_hist[str(months_official[month])]=np.append(occurence_hist[str(months_official[month])],(occurence_hist['all'][entry,:]))
                    occurence_fut[str(months_official[month])]=np.append(occurence_fut[str(months_official[month])],(occurence_fut['all'][entry,:]))
            occurence_hist[str(months_official[month])]= np.reshape(occurence_hist[str(months_official[month])], (int(len(occurence_hist[str(months_official[month])])/6),6))
            occurence_fut[str(months_official[month])]= np.reshape(occurence_fut[str(months_official[month])], (int(len(occurence_fut[str(months_official[month])])/6),6))
            counts_hist[str(months_official[month])]=np.bincount(np.round(occurence_hist[str(months_official[month])][:,3]).astype(int))/(years*month_len[month])
            counts_fut[str(months_official[month])]=np.bincount(np.round(occurence_fut[str(months_official[month])][:,3]).astype(int))/(years*month_len[month]) #bei zweimodellen gibt es 120 0zuweisungen
            if counts_hist[str(months_official[month])][0]>0:
                print(models[model]+" da sind 120 falschzuweisungen")
        for month in range(0,months):
            if model==0:
                difference_cl1[str(months_official[month])]=np.array([])
                difference_cl2[str(months_official[month])]=np.array([])
                difference_cl3[str(months_official[month])]=np.array([])
                difference_cl4[str(months_official[month])]=np.array([])
                difference_cl5[str(months_official[month])]=np.array([])
            else:
                pass
            difference_cl1[str(months_official[month])]=np.append(difference_cl1[str(months_official[month])],counts_fut[str(months_official[month])][assignment_fut['CL1/SCAN'][model]]-counts_hist[str(months_official[month])][assignment_hist['CL1/SCAN'][model]])
            difference_cl2[str(months_official[month])]=np.append(difference_cl2[str(months_official[month])],counts_fut[str(months_official[month])][assignment_fut['CL2/NAO+'][model]]-counts_hist[str(months_official[month])][assignment_hist['CL2/NAO+'][model]])
            difference_cl3[str(months_official[month])]=np.append(difference_cl3[str(months_official[month])],counts_fut[str(months_official[month])][assignment_fut['CL3/NAO-'][model]]-counts_hist[str(months_official[month])][assignment_hist['CL3/NAO-'][model]])
            difference_cl4[str(months_official[month])]=np.append(difference_cl4[str(months_official[month])],counts_fut[str(months_official[month])][assignment_fut['CL4/DIPOL'][model]]-counts_hist[str(months_official[month])][assignment_hist['CL4/DIPOL'][model]])
            difference_cl5[str(months_official[month])]=np.append(difference_cl5[str(months_official[month])],counts_fut[str(months_official[month])][assignment_fut['CL5/ATL'][model]]-counts_hist[str(months_official[month])][assignment_hist['CL5/ATL'][model]])
        if model==0:
            difference_cl1['all']=np.array([])
            difference_cl2['all']=np.array([])
            difference_cl3['all']=np.array([])
            difference_cl4['all']=np.array([])
            difference_cl5['all']=np.array([])
        else:
            pass
        difference_cl1['all']=np.append(difference_cl1['all'],float(f"{counts_fut['all'][assignment_fut['CL1/SCAN'][model]]-counts_hist['all'][assignment_hist['CL1/SCAN'][model]]:.2f}"))
        difference_cl2['all']=np.append(difference_cl2['all'],float(f"{counts_fut['all'][assignment_fut['CL2/NAO+'][model]]-counts_hist['all'][assignment_hist['CL2/NAO+'][model]]:.2f}"))
        difference_cl3['all']=np.append(difference_cl3['all'],float(f"{counts_fut['all'][assignment_fut['CL3/NAO-'][model]]-counts_hist['all'][assignment_hist['CL3/NAO-'][model]]:.2f}"))
        difference_cl4['all']=np.append(difference_cl4['all'],float(f"{counts_fut['all'][assignment_fut['CL4/DIPOL'][model]]-counts_hist['all'][assignment_hist['CL4/DIPOL'][model]]:.2f}"))
        difference_cl5['all']=np.append(difference_cl5['all'],float(f"{counts_fut['all'][assignment_fut['CL5/ATL'][model]]-counts_hist['all'][assignment_hist['CL5/ATL'][model]]:.2f}"))
        #significance: spalten sind die cl, zeilen die monate, alles in 6x5 arrays
        #minmax: zuerst 5 cl dann nächster monat dann 5 cl dann nächster monat von cl1 nach cl5, alles in 6x5 arrays
        table_difference_cl1 = pd.DataFrame.from_dict(difference_cl1, orient='index', columns=models[:model+1]).transpose()
        table_difference_cl2 = pd.DataFrame.from_dict(difference_cl2, orient='index', columns=models[:model+1]).transpose()
        table_difference_cl3 = pd.DataFrame.from_dict(difference_cl3, orient='index', columns=models[:model+1]).transpose()
        table_difference_cl4 = pd.DataFrame.from_dict(difference_cl4, orient='index', columns=models[:model+1]).transpose()
        table_difference_cl5 = pd.DataFrame.from_dict(difference_cl5, orient='index', columns=models[:model+1]).transpose()
        #break()
        #hier durchschnittbilden
        if model==len(models)-1:
            difference_cl1['all']=np.append(difference_cl1['all'], np.mean(difference_cl1['all']))
            difference_cl2['all']=np.append(difference_cl2['all'], np.mean(difference_cl2['all']))
            difference_cl3['all']=np.append(difference_cl3['all'], np.mean(difference_cl3['all']))
            difference_cl4['all']=np.append(difference_cl4['all'], np.mean(difference_cl4['all']))
            difference_cl5['all']=np.append(difference_cl5['all'], np.mean(difference_cl5['all']))                           
            for month in range(0,months):
                difference_cl1[str(months_official[month])]=np.append(difference_cl1[str(months_official[month])],np.mean(difference_cl1[str(months_official[month])]))
                difference_cl2[str(months_official[month])]=np.append(difference_cl2[str(months_official[month])],np.mean(difference_cl2[str(months_official[month])]))
                difference_cl3[str(months_official[month])]=np.append(difference_cl3[str(months_official[month])],np.mean(difference_cl3[str(months_official[month])]))
                difference_cl4[str(months_official[month])]=np.append(difference_cl4[str(months_official[month])],np.mean(difference_cl4[str(months_official[month])]))
                difference_cl5[str(months_official[month])]=np.append(difference_cl5[str(months_official[month])],np.mean(difference_cl5[str(months_official[month])]))
            models.append('add')
            table_difference_cl1 = pd.DataFrame.from_dict(difference_cl1, orient='index', columns=models[:model+2]).transpose().round(2)
            table_difference_cl2 = pd.DataFrame.from_dict(difference_cl2, orient='index', columns=models[:model+2]).transpose().round(2)
            table_difference_cl3 = pd.DataFrame.from_dict(difference_cl3, orient='index', columns=models[:model+2]).transpose().round(2)
            table_difference_cl4 = pd.DataFrame.from_dict(difference_cl4, orient='index', columns=models[:model+2]).transpose().round(2)
            table_difference_cl5 = pd.DataFrame.from_dict(difference_cl5, orient='index', columns=models[:model+2]).transpose().round(2)  
            models=models[:-2]
            
            
            table_difference_cl1_styled=table_difference_cl1.style.background_gradient(axis=None, vmin=-0.2, vmax=0.2, cmap="seismic").format('{:.2f}')
            table_difference_cl2_styled=table_difference_cl2.style.background_gradient(axis=None, vmin=-0.2, vmax=0.2, cmap="seismic").format('{:.2f}')
            table_difference_cl3_styled=table_difference_cl3.style.background_gradient(axis=None, vmin=-0.2, vmax=0.2, cmap="seismic").format('{:.2f}')
            table_difference_cl4_styled=table_difference_cl4.style.background_gradient(axis=None, vmin=-0.2, vmax=0.2, cmap="seismic").format('{:.2f}')
            table_difference_cl5_styled=table_difference_cl5.style.background_gradient(axis=None, vmin=-0.2, vmax=0.2, cmap="seismic").format('{:.2f}')
        #durchschnitt bilden vorbeistyler.format('{:.2f}')
        #jetzt die farben
       #for month in range(0,months): 
        #    styled_df = df.style.applymap(lambda x: significance_decider(x, significance[month,]))
            print(table_difference_cl1_styled)
            print(table_difference_cl2_styled)
            print(table_difference_cl3_styled)
            print(table_difference_cl4_styled)
            print(table_difference_cl5_styled)
            # render the styled dataframe as HTML with CSS formatting
            html = table_difference_cl1_styled.render()
            css = re.findall('style=\"([\w\W]*?)\"', html)
            table_difference_cl1_colored = pd.DataFrame(table_difference_cl1.values, index=table_difference_cl1.index, columns=table_difference_cl1.columns)
            for i, row in enumerate(css):
                for j, col in enumerate(row.split(';')):
                    if 'background-color' in col:
                        table_difference_cl1_colored.iloc[i, j] = col.split(':')[1]
            latex_table = table_difference_cl1_colored.to_latex(index=False)
            with open('N:/atm_glomod/user/jomuel001/auswertung/BOOTSTRAP/Daten_CMIP6/table_cl1.tex', 'w') as f:
                f.write(latex_table)

            html = table_difference_cl2_styled.render()
            css = re.findall('style=\"([\w\W]*?)\"', html)
            table_difference_cl2_colored = pd.DataFrame(table_difference_cl2.values, index=table_difference_cl2.index, columns=table_difference_cl2.columns)
            for i, row in enumerate(css):
                for j, col in enumerate(row.split(';')):
                    if 'background-color' in col:
                        table_difference_cl2_colored.iloc[i, j] = col.split(':')[1]
            latex_table = table_difference_cl2_colored.to_latex(index=False)
            with open('N:/atm_glomod/user/jomuel001/auswertung/BOOTSTRAP/Daten_CMIP6/table_cl2.tex', 'w') as f:
                f.write(latex_table)

            html = table_difference_cl3_styled.render()
            css = re.findall('style=\"([\w\W]*?)\"', html)
            table_difference_cl3_colored = pd.DataFrame(table_difference_cl3.values, index=table_difference_cl3.index, columns=table_difference_cl3.columns)
            for i, row in enumerate(css):
                for j, col in enumerate(row.split(';')):
                    if 'background-color' in col:
                        table_difference_cl3_colored.iloc[i, j] = col.split(':')[1]
            latex_table = table_difference_cl3_colored.to_latex(index=False)
            with open('N:/atm_glomod/user/jomuel001/auswertung/BOOTSTRAP/Daten_CMIP6/table_cl3.tex', 'w') as f:
                f.write(latex_table)

            html = table_difference_cl4_styled.render()
            css = re.findall('style=\"([\w\W]*?)\"', html)
            table_difference_cl4_colored = pd.DataFrame(table_difference_cl4.values, index=table_difference_cl4.index, columns=table_difference_cl4.columns)
            for i, row in enumerate(css):
                for j, col in enumerate(row.split(';')):
                    if 'background-color' in col:
                        table_difference_cl4_colored.iloc[i, j] = col.split(':')[1]
            latex_table = table_difference_cl4_colored.to_latex(index=False)
            with open('N:/atm_glomod/user/jomuel001/auswertung/BOOTSTRAP/Daten_CMIP6/table_cl4.tex', 'w') as f:
                f.write(latex_table)

            html = table_difference_cl5_styled.render()
            css = re.findall('style=\"([\w\W]*?)\"', html)
            table_difference_cl5_colored = pd.DataFrame(table_difference_cl5.values, index=table_difference_cl5.index, columns=table_difference_cl5.columns)
            for i, row in enumerate(css):
                for j, col in enumerate(row.split(';')):
                    if 'background-color' in col:
                        table_difference_cl5_colored.iloc[i, j] = col.split(':')[1]
            latex_table = table_difference_cl5_colored.to_latex(index=False)
            with open('N:/atm_glomod/user/jomuel001/auswertung/BOOTSTRAP/Daten_CMIP6/table_cl5.tex', 'w') as f:
                f.write(latex_table)
                
#####################################################################################################################################################################################################################################                   
#####################################################################################################################################################################################################################################
#####################################################################################################################################################################################################################################                   
#####################################################################################################################################################################################################################################
#####################################################################################################################################################################################################################################                   
#####################################################################################################################################################################################################################################

elif aufgabe == 'Projected patterns':
    for model in range(len(models)):
        path_2="N:/atm_glomod/user/jomuel001/CMIP6_models/"+models[model]+"/AREA."+area+"_89.7849_29.0866/CLUSTER/"
        path="N:/atm_glomod/user/jomuel001/CMIP6_models/"+models[model]+"/AREA."+area+"_89.7849_29.0866/CLUSTER/PLOTS/"
        datei_signi=r"KMEANSslp_hpa_"+models[model]+"_"+years_hist+".N_"+seas+"_atrbg_aacrm21_remapbnds_proj_on_EOF_ERA5"+years_proj+"slp_hpa_ERA5_"+years_proj+"_"+seas+"_atrbg_aacrm21_remapbnds_unnorm_projectedpattern_5cluster10PC_high_low.month.signif_bootstrap.txt"
        datei_minmax=r"KMEANSslp_hpa_"+models[model]+"_"+years_hist+".N_"+seas+"_atrbg_aacrm21_remapbnds_proj_on_EOF_ERA5"+years_proj+"slp_hpa_ERA5_"+years_proj+"_"+seas+"_atrbg_aacrm21_remapbnds_unnorm_projectedpattern_5cluster10PC_high_low.month.signif_bootstrap_minmaxwhiskers.txt"
        #cluster werden zu era5 zugeordnet
        datei_occ_hist=r"KMEANSslp_hpa_"+models[model]+"_"+years_hist+".N_"+seas+"_atrbg_aacrm21_remapbnds_proj_on_EOF_ERA5"+years_proj+"slp_hpa_ERA5_"+years_proj+"_"+seas+"_atrbg_aacrm21_remapbnds_unnorm_projectedpattern_5tabcluster_10tabPC_dates"
        datei_occ_fut=r"KMEANSslp_hpa_"+models[model]+"_"+years_fut+".N_"+seas+"_atrbg_aacHrm21_remapbnds_proj_on_EOF_ERA5"+years_proj+"slp_hpa_ERA5_"+years_proj+"_"+seas+"_atrbg_aacrm21_remapbnds_unnorm_projectedpattern_5tabcluster_10tabPC_dates"

        significance=np.array(pd.read_csv(path+datei_signi, delimiter=' '))[~np.isnan(np.array(pd.read_csv(path+datei_signi, delimiter=' ')))].reshape(months,5)
        minmaxwhiskers=np.array(pd.read_csv(path+datei_minmax, delimiter=' '))[~np.isnan(np.array(pd.read_csv(path+datei_minmax, delimiter=' ')))].reshape(months,5,6)
        occurence_hist['all']=np.loadtxt(path_2+datei_occ_hist, delimiter=' ')
        occurence_fut['all']=np.loadtxt(path_2+datei_occ_fut, delimiter=' ')
       # print(occurence_hist['all'])
       # print(occurence_fut['all'])
        counts_hist['all']=np.bincount(np.round(occurence_hist['all'][:,3]).astype(int))/(days_per_period)
        counts_fut['all']=np.bincount(np.round(occurence_fut['all'][:,3]).astype(int))/(days_per_period)
        #print(counts_hist['all'])
        #print(counts_fut['all'])

        for month in range(0,months):
            occurence_hist[str(months_official[month])]=np.array([])
            occurence_fut[str(months_official[month])]=np.array([])
            for entry in range(len(occurence_hist['all'])):
                if occurence_hist['all'][entry,1]==months_name[month]:
                    occurence_hist[str(months_official[month])]=np.append(occurence_hist[str(months_official[month])],(occurence_hist['all'][entry,:]))
                    occurence_fut[str(months_official[month])]=np.append(occurence_fut[str(months_official[month])],(occurence_fut['all'][entry,:]))
            occurence_hist[str(months_official[month])]= np.reshape(occurence_hist[str(months_official[month])], (int(len(occurence_hist[str(months_official[month])])/6),6))
            occurence_fut[str(months_official[month])]= np.reshape(occurence_fut[str(months_official[month])], (int(len(occurence_fut[str(months_official[month])])/6),6))
            #print(occurence_hist['May'])
            #print(occurence_fut['May'])
            counts_hist[str(months_official[month])]=np.bincount(np.round(occurence_hist[str(months_official[month])][:,3]).astype(int))/(years*month_len[month])
            counts_fut[str(months_official[month])]=np.bincount(np.round(occurence_fut[str(months_official[month])][:,3]).astype(int))/(years*month_len[month]) #bei zweimodellen gibt es 120 0zuweisungen
            #print(years*month_len[month])
            #print(counts_hist['May'])
            #print(counts_fut['May'])
            if counts_hist[str(months_official[month])][0]>0:
                print(models[model]+" da sind 120 falschzuweisungen")
        for month in range(0,months):
            if model==0:
                difference_cl1[str(months_official[month])]=np.array([])
                difference_cl2[str(months_official[month])]=np.array([])
                difference_cl3[str(months_official[month])]=np.array([])
                difference_cl4[str(months_official[month])]=np.array([])
                difference_cl5[str(months_official[month])]=np.array([])
            else:
                pass
            difference_cl1[str(months_official[month])]=np.append(difference_cl1[str(months_official[month])],counts_fut[str(months_official[month])][1]-counts_hist[str(months_official[month])][1])
            difference_cl2[str(months_official[month])]=np.append(difference_cl2[str(months_official[month])],counts_fut[str(months_official[month])][2]-counts_hist[str(months_official[month])][2])
            difference_cl3[str(months_official[month])]=np.append(difference_cl3[str(months_official[month])],counts_fut[str(months_official[month])][3]-counts_hist[str(months_official[month])][3])
            difference_cl4[str(months_official[month])]=np.append(difference_cl4[str(months_official[month])],counts_fut[str(months_official[month])][4]-counts_hist[str(months_official[month])][4])
            difference_cl5[str(months_official[month])]=np.append(difference_cl5[str(months_official[month])],counts_fut[str(months_official[month])][5]-counts_hist[str(months_official[month])][5])
        if model==0:
            difference_cl1['all']=np.array([])
            difference_cl2['all']=np.array([])
            difference_cl3['all']=np.array([])
            difference_cl4['all']=np.array([])
            difference_cl5['all']=np.array([])
        else:
            pass
        #print(counts_hist['all'])
        #print(counts_fut['all'])
        #print(counts_hist['June'])
        #print(counts_fut['June'])
        difference_cl1['all']=np.append(difference_cl1['all'],float(f"{counts_fut['all'][1]-counts_hist['all'][1]:.2f}"))
        #print(difference_cl1['all'])
        difference_cl2['all']=np.append(difference_cl2['all'],float(f"{counts_fut['all'][2]-counts_hist['all'][2]:.2f}"))
        difference_cl3['all']=np.append(difference_cl3['all'],float(f"{counts_fut['all'][3]-counts_hist['all'][3]:.2f}"))
        difference_cl4['all']=np.append(difference_cl4['all'],float(f"{counts_fut['all'][4]-counts_hist['all'][4]:.2f}"))
        difference_cl5['all']=np.append(difference_cl5['all'],float(f"{counts_fut['all'][5]-counts_hist['all'][5]:.2f}"))
        #significance: spalten sind die cl, zeilen die monate, alles in 6x5 arrays
        #minmax: zuerst 5 cl dann nächster monat dann 5 cl dann nächster monat von cl1 nach cl5, alles in 6x5 arrays
        table_difference_cl1 = pd.DataFrame.from_dict(difference_cl1, orient='index', columns=models[:model+1]).transpose()
        table_difference_cl2 = pd.DataFrame.from_dict(difference_cl2, orient='index', columns=models[:model+1]).transpose()
        table_difference_cl3 = pd.DataFrame.from_dict(difference_cl3, orient='index', columns=models[:model+1]).transpose()
        table_difference_cl4 = pd.DataFrame.from_dict(difference_cl4, orient='index', columns=models[:model+1]).transpose()
        table_difference_cl5 = pd.DataFrame.from_dict(difference_cl5, orient='index', columns=models[:model+1]).transpose()
        #hier durchschnittbilden
        for month in range(0,months):
            column_name=str(months_official[month])
            row_index=model
            style_dict_cl1[column_name] = np.append(style_dict_cl1[column_name], significance_decider(table_difference_cl1[column_name][row_index], significance[month,1-1], minmaxwhiskers[month][1-1]))
            style_dict_cl2[column_name] = np.append(style_dict_cl2[column_name], significance_decider(table_difference_cl2[column_name][row_index], significance[month,2-1], minmaxwhiskers[month][2-1]))
            style_dict_cl3[column_name] = np.append(style_dict_cl3[column_name], significance_decider(table_difference_cl3[column_name][row_index], significance[month,3-1], minmaxwhiskers[month][3-1]))
            style_dict_cl4[column_name] = np.append(style_dict_cl4[column_name], significance_decider(table_difference_cl4[column_name][row_index], significance[month,4-1], minmaxwhiskers[month][4-1]))
            style_dict_cl5[column_name] = np.append(style_dict_cl5[column_name], significance_decider(table_difference_cl5[column_name][row_index], significance[month,5-1], minmaxwhiskers[month][5-1]))
            #print(style_dict)
        style_dict_cl1['all']=np.append(style_dict_cl1['all'], significance_decider(table_difference_cl1[column_name][row_index], 0, minmaxwhiskers[month][1-1]))
        style_dict_cl2['all']=np.append(style_dict_cl2['all'], significance_decider(table_difference_cl2[column_name][row_index], 0, minmaxwhiskers[month][2-1]))
        style_dict_cl3['all']=np.append(style_dict_cl3['all'], significance_decider(table_difference_cl3[column_name][row_index], 0, minmaxwhiskers[month][3-1]))
        style_dict_cl4['all']=np.append(style_dict_cl4['all'], significance_decider(table_difference_cl4[column_name][row_index], 0, minmaxwhiskers[month][4-1]))
        style_dict_cl5['all']=np.append(style_dict_cl5['all'], significance_decider(table_difference_cl5[column_name][row_index], 0, minmaxwhiskers[month][5-1]))
            #table_difference_cl1_styled=table_difference_cl1.style.applymap(lambda val: significance_decider(val, significance[month,1-1], minmaxwhiskers[month][1-1]) if (val == table_difference_cl1[column_name][row_index]) else '')
        if model==len(models)-1:
            difference_cl1['all']=np.append(difference_cl1['all'], np.mean(difference_cl1['all']))
            difference_cl2['all']=np.append(difference_cl2['all'], np.mean(difference_cl2['all']))
            difference_cl3['all']=np.append(difference_cl3['all'], np.mean(difference_cl3['all']))
            difference_cl4['all']=np.append(difference_cl4['all'], np.mean(difference_cl4['all']))
            difference_cl5['all']=np.append(difference_cl5['all'], np.mean(difference_cl5['all']))                           
            for month in range(0,months):
                difference_cl1[str(months_official[month])]=np.append(difference_cl1[str(months_official[month])],np.mean(difference_cl1[str(months_official[month])]))
                difference_cl2[str(months_official[month])]=np.append(difference_cl2[str(months_official[month])],np.mean(difference_cl2[str(months_official[month])]))
                difference_cl3[str(months_official[month])]=np.append(difference_cl3[str(months_official[month])],np.mean(difference_cl3[str(months_official[month])]))
                difference_cl4[str(months_official[month])]=np.append(difference_cl4[str(months_official[month])],np.mean(difference_cl4[str(months_official[month])]))
                difference_cl5[str(months_official[month])]=np.append(difference_cl5[str(months_official[month])],np.mean(difference_cl5[str(months_official[month])]))
            models.append('add')
            table_difference_cl1 = pd.DataFrame.from_dict(difference_cl1, orient='index', columns=models[:model+2]).transpose().round(2)
            table_difference_cl2 = pd.DataFrame.from_dict(difference_cl2, orient='index', columns=models[:model+2]).transpose().round(2)
            table_difference_cl3 = pd.DataFrame.from_dict(difference_cl3, orient='index', columns=models[:model+2]).transpose().round(2)
            table_difference_cl4 = pd.DataFrame.from_dict(difference_cl4, orient='index', columns=models[:model+2]).transpose().round(2)
            table_difference_cl5 = pd.DataFrame.from_dict(difference_cl5, orient='index', columns=models[:model+2]).transpose().round(2)  
            for month in range(0,months):
                column_name=str(months_official[month])
                row_index=model+1
                style_dict_cl1[column_name] = np.append(style_dict_cl1[column_name], significance_decider(table_difference_cl1[column_name][row_index], 0, minmaxwhiskers[month][1-1]))
                style_dict_cl2[column_name] = np.append(style_dict_cl2[column_name], significance_decider(table_difference_cl2[column_name][row_index], 0, minmaxwhiskers[month][2-1]))
                style_dict_cl3[column_name] = np.append(style_dict_cl3[column_name], significance_decider(table_difference_cl3[column_name][row_index], 0, minmaxwhiskers[month][3-1]))
                style_dict_cl4[column_name] = np.append(style_dict_cl4[column_name], significance_decider(table_difference_cl4[column_name][row_index], 0, minmaxwhiskers[month][4-1]))
                style_dict_cl5[column_name] = np.append(style_dict_cl5[column_name], significance_decider(table_difference_cl5[column_name][row_index], 0, minmaxwhiskers[month][5-1]))
            #print(style_dict)
            style_dict_cl1['all']=np.append(style_dict_cl1['all'], significance_decider(table_difference_cl1[column_name][row_index], 0, minmaxwhiskers[month][1-1]))
            style_dict_cl2['all']=np.append(style_dict_cl2['all'], significance_decider(table_difference_cl2[column_name][row_index], 0, minmaxwhiskers[month][2-1]))
            style_dict_cl3['all']=np.append(style_dict_cl3['all'], significance_decider(table_difference_cl3[column_name][row_index], 0, minmaxwhiskers[month][3-1]))
            style_dict_cl4['all']=np.append(style_dict_cl4['all'], significance_decider(table_difference_cl4[column_name][row_index], 0, minmaxwhiskers[month][4-1]))
            style_dict_cl5['all']=np.append(style_dict_cl5['all'], significance_decider(table_difference_cl5[column_name][row_index], 0, minmaxwhiskers[month][5-1]))
            models=models[:-2]
            
            table_difference_cl1_styled=table_difference_cl1.style.background_gradient(axis=None, vmin=-0.2, vmax=0.2, cmap="seismic").format('{:.2f}')
            table_difference_cl2_styled=table_difference_cl2.style.background_gradient(axis=None, vmin=-0.2, vmax=0.2, cmap="seismic").format('{:.2f}')
            table_difference_cl3_styled=table_difference_cl3.style.background_gradient(axis=None, vmin=-0.2, vmax=0.2, cmap="seismic").format('{:.2f}')
            table_difference_cl4_styled=table_difference_cl4.style.background_gradient(axis=None, vmin=-0.2, vmax=0.2, cmap="seismic").format('{:.2f}')
            table_difference_cl5_styled=table_difference_cl5.style.background_gradient(axis=None, vmin=-0.2, vmax=0.2, cmap="seismic").format('{:.2f}')
        #durchschnitt bilden vorbeistyler.format('{:.2f}')
        #jetzt die farben
       #for month in range(0,months): 
        #    styled_df = df.style.applymap(lambda x: significance_decider(x, significance[month,]))

            # render the styled dataframe as HTML with CSS formatting

            html = table_difference_cl1_styled.render()
            css = re.findall('style=\"([\w\W]*?)\"', html)
            table_difference_cl1_colored = pd.DataFrame(table_difference_cl1.values, index=table_difference_cl1.index, columns=table_difference_cl1.columns)
            for i, row in enumerate(css):
                for j, col in enumerate(row.split(';')):
                    if 'background-color' in col:
                        table_difference_cl1_colored.iloc[i, j] = col.split(':')[1]
            latex_table = table_difference_cl1_colored.to_latex(index=False)
            with open('N:/atm_glomod/user/jomuel001/auswertung/BOOTSTRAP/Daten_CMIP6/table_cl1_proj.tex', 'w') as f:
                f.write(latex_table)

            html = table_difference_cl2_styled.render()
            css = re.findall('style=\"([\w\W]*?)\"', html)
            table_difference_cl2_colored = pd.DataFrame(table_difference_cl2.values, index=table_difference_cl2.index, columns=table_difference_cl2.columns)
            for i, row in enumerate(css):
                for j, col in enumerate(row.split(';')):
                    if 'background-color' in col:
                        table_difference_cl2_colored.iloc[i, j] = col.split(':')[1]
            latex_table = table_difference_cl2_colored.to_latex(index=False)
            with open('N:/atm_glomod/user/jomuel001/auswertung/BOOTSTRAP/Daten_CMIP6/table_cl2_proj.tex', 'w') as f:
                f.write(latex_table)

            html = table_difference_cl3_styled.render()
            css = re.findall('style=\"([\w\W]*?)\"', html)
            table_difference_cl3_colored = pd.DataFrame(table_difference_cl3.values, index=table_difference_cl3.index, columns=table_difference_cl3.columns)
            for i, row in enumerate(css):
                for j, col in enumerate(row.split(';')):
                    if 'background-color' in col:
                        table_difference_cl3_colored.iloc[i, j] = col.split(':')[1]
            latex_table = table_difference_cl3_colored.to_latex(index=False)
            with open('N:/atm_glomod/user/jomuel001/auswertung/BOOTSTRAP/Daten_CMIP6/table_cl3_proj.tex', 'w') as f:
                f.write(latex_table)

            html = table_difference_cl4_styled.render()
            css = re.findall('style=\"([\w\W]*?)\"', html)
            table_difference_cl4_colored = pd.DataFrame(table_difference_cl4.values, index=table_difference_cl4.index, columns=table_difference_cl4.columns)
            for i, row in enumerate(css):
                for j, col in enumerate(row.split(';')):
                    if 'background-color' in col:
                        table_difference_cl4_colored.iloc[i, j] = col.split(':')[1]
            latex_table = table_difference_cl4_colored.to_latex(index=False)
            with open('N:/atm_glomod/user/jomuel001/auswertung/BOOTSTRAP/Daten_CMIP6/table_cl4_proj.tex', 'w') as f:
                f.write(latex_table)

            html = table_difference_cl5_styled.render()
            css = re.findall('style=\"([\w\W]*?)\"', html)
            table_difference_cl5_colored = pd.DataFrame(table_difference_cl5.values, index=table_difference_cl5.index, columns=table_difference_cl5.columns)
            for i, row in enumerate(css):
                for j, col in enumerate(row.split(';')):
                    if 'background-color' in col:
                        table_difference_cl5_colored.iloc[i, j] = col.split(':')[1]
            latex_table = table_difference_cl5_colored.to_latex(index=False)
            with open('N:/atm_glomod/user/jomuel001/auswertung/BOOTSTRAP/Daten_CMIP6/table_cl5_proj.tex', 'w') as f:
                f.write(latex_table)
            #cluster1 ist hier 
    else: 
        print("verschrieben")
    display(table_difference_cl1_styled)
    display(table_difference_cl2_styled)
    display(table_difference_cl3_styled)
    display(table_difference_cl4_styled)
    display(table_difference_cl5_styled)
    


            


FileNotFoundError: [Errno 2] No such file or directory: 'N:/atm_glomod/user/jomuel001/CMIP6_models/KACE-1-0-G/AREA.-90_90_89.7849_29.0866/CLUSTER/PLOTS/KMEANSslp_hpa_KACE-1-0-G_1985-2014.N_mjjaso_atrbg_aacrm21_remapbnds_proj_on_EOF_ERA51985-2014slp_hpa_ERA5_1985-2014_mjjaso_atrbg_aacrm21_remapbnds_unnorm_projectedpattern_5cluster10PC_high_low.month.signif_bootstrap.txt'

# Projected patterns - mit signifikanz bemalung

In [174]:
models =["ACCESS-CM2", "ACCESS-ESM1", "CESM2-WACCM", "CNRM-CM6-1", "CNRM-CM6-1-HR", "CNRM-ESM2-1", 
         "GFDL-CM4", "GFDL-ESM4","HadGEM3-GC31-LL", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR", "MIROC6", 
         "MPI-ESM1-2-HR", "MPI-ESM1-2-LR", "MRI-ESM2-0", "NorESM2-MM", "UKESM1-0-LL"]
style_df_cl1 = pd.DataFrame.from_dict(style_dict_cl1, orient='index', columns=models+['add']).transpose().round(2)
style_df_cl2 = pd.DataFrame.from_dict(style_dict_cl2, orient='index', columns=models+['add']).transpose().round(2)
style_df_cl3 = pd.DataFrame.from_dict(style_dict_cl3, orient='index', columns=models+['add']).transpose().round(2)
style_df_cl4 = pd.DataFrame.from_dict(style_dict_cl4, orient='index', columns=models+['add']).transpose().round(2)
style_df_cl5 = pd.DataFrame.from_dict(style_dict_cl5, orient='index', columns=models+['add']).transpose().round(2)
table_difference_cl1_styled_signi = table_difference_cl1.style.set_caption("Clusterpattern 1")
table_difference_cl2_styled_signi = table_difference_cl2.style.set_caption("Clusterpattern 2")
table_difference_cl3_styled_signi = table_difference_cl3.style.set_caption("Clusterpattern 3")
table_difference_cl4_styled_signi = table_difference_cl4.style.set_caption("Clusterpattern 4")
table_difference_cl5_styled_signi = table_difference_cl5.style.set_caption("Clusterpattern 5")
for idx_r in range(0,19):
    for idx_c in range(0,7):
        table_difference_cl1_styled_signi.apply(styling_specific_cell1, row_idx = idx_r, col_idx = idx_c, axis = None).format('{:.2f}')
        table_difference_cl2_styled_signi.apply(styling_specific_cell2, row_idx = idx_r, col_idx = idx_c, axis = None).format('{:.2f}')
        table_difference_cl3_styled_signi.apply(styling_specific_cell3, row_idx = idx_r, col_idx = idx_c, axis = None).format('{:.2f}')
        table_difference_cl4_styled_signi.apply(styling_specific_cell4, row_idx = idx_r, col_idx = idx_c, axis = None).format('{:.2f}')
        table_difference_cl5_styled_signi.apply(styling_specific_cell5, row_idx = idx_r, col_idx = idx_c, axis = None).format('{:.2f}')
        
display(table_difference_cl1_styled_signi)
display(table_difference_cl2_styled_signi)
display(table_difference_cl3_styled_signi)
display(table_difference_cl4_styled_signi)
display(table_difference_cl5_styled_signi)

,May,June,July,August,September,October,all
ACCESS-CM2,-0.04,-0.07,-0.06,-0.10,-0.09,-0.09,-0.07
ACCESS-ESM1,-0.05,-0.01,0.00,-0.03,-0.03,-0.04,-0.03
CESM2-WACCM,-0.04,0.13,0.11,0.05,0.06,-0.11,0.03
CNRM-CM6-1,0.00,-0.16,-0.12,-0.11,0.00,-0.05,-0.07
CNRM-CM6-1-HR,0.01,-0.05,-0.15,-0.09,-0.06,0.03,-0.05
CNRM-ESM2-1,-0.06,-0.06,-0.07,-0.02,-0.00,-0.08,-0.05
GFDL-CM4,0.05,-0.01,-0.04,-0.01,-0.01,-0.06,-0.01
GFDL-ESM4,-0.01,-0.07,-0.03,0.02,0.03,-0.09,-0.03
HadGEM3-GC31-LL,-0.05,-0.05,-0.08,-0.08,-0.06,-0.04,-0.06
INM-CM4-8,-0.02,-0.07,-0.09,-0.10,-0.01,-0.05,-0.06


,May,June,July,August,September,October,all
ACCESS-CM2,-0.01,-0.03,0.01,0.07,-0.04,0.11,0.02
ACCESS-ESM1,-0.01,-0.03,-0.03,-0.07,-0.08,0.02,-0.03
CESM2-WACCM,-0.04,-0.21,-0.09,-0.05,-0.07,-0.02,-0.08
CNRM-CM6-1,0.03,-0.06,0.02,-0.05,-0.06,0.04,-0.01
CNRM-CM6-1-HR,-0.04,0.06,0.12,0.05,0.03,0.00,0.04
CNRM-ESM2-1,-0.03,-0.01,-0.00,0.03,-0.07,0.03,-0.01
GFDL-CM4,-0.06,-0.09,-0.09,-0.06,-0.04,0.02,-0.05
GFDL-ESM4,-0.07,-0.00,-0.09,-0.06,-0.11,0.05,-0.05
HadGEM3-GC31-LL,-0.06,-0.03,-0.10,-0.09,-0.07,-0.09,-0.07
INM-CM4-8,-0.01,0.02,-0.01,-0.02,-0.06,0.01,-0.01


,May,June,July,August,September,October,all
ACCESS-CM2,0.06,0.05,0.09,0.12,0.09,0.04,0.07
ACCESS-ESM1,0.06,-0.05,0.06,0.12,0.08,0.06,0.06
CESM2-WACCM,0.10,-0.04,-0.03,0.04,0.09,0.09,0.04
CNRM-CM6-1,0.07,0.11,0.07,0.18,0.10,0.06,0.10
CNRM-CM6-1-HR,-0.02,0.06,0.00,0.08,0.15,0.05,0.05
CNRM-ESM2-1,0.04,0.02,-0.00,0.00,0.08,0.15,0.05
GFDL-CM4,0.02,0.02,-0.01,0.11,-0.05,-0.04,0.01
GFDL-ESM4,0.02,0.06,0.11,-0.02,0.09,0.06,0.05
HadGEM3-GC31-LL,0.03,0.11,0.13,0.22,0.09,0.08,0.11
INM-CM4-8,-0.04,0.03,0.02,0.08,0.05,0.08,0.04


,May,June,July,August,September,October,all
ACCESS-CM2,-0.02,-0.06,-0.08,-0.15,-0.08,-0.12,-0.08
ACCESS-ESM1,0.00,0.05,-0.03,-0.05,-0.05,-0.05,-0.02
CESM2-WACCM,-0.08,0.10,-0.01,-0.08,-0.09,-0.04,-0.03
CNRM-CM6-1,-0.12,-0.01,-0.07,-0.10,-0.08,-0.11,-0.08
CNRM-CM6-1-HR,-0.00,-0.09,-0.02,-0.07,-0.12,-0.08,-0.06
CNRM-ESM2-1,0.04,0.03,-0.03,-0.08,-0.04,-0.10,-0.03
GFDL-CM4,-0.04,0.01,0.00,-0.10,0.01,-0.03,-0.02
GFDL-ESM4,-0.02,-0.08,-0.06,-0.04,-0.03,-0.08,-0.05
HadGEM3-GC31-LL,-0.00,-0.04,-0.09,-0.10,-0.08,-0.05,-0.06
INM-CM4-8,-0.01,-0.12,-0.01,-0.09,-0.01,-0.06,-0.05


,May,June,July,August,September,October,all
ACCESS-CM2,0.02,0.11,0.04,0.05,0.13,0.06,0.07
ACCESS-ESM1,0.00,0.03,-0.01,0.04,0.07,0.01,0.02
CESM2-WACCM,0.07,0.02,0.02,0.04,0.01,0.08,0.04
CNRM-CM6-1,0.02,0.11,0.10,0.08,0.05,0.05,0.07
CNRM-CM6-1-HR,0.06,0.03,0.05,0.03,0.00,-0.01,0.03
CNRM-ESM2-1,0.00,0.02,0.10,0.05,0.02,0.01,0.04
GFDL-CM4,0.03,0.07,0.13,0.05,0.08,0.10,0.08
GFDL-ESM4,0.08,0.09,0.08,0.10,0.02,0.08,0.07
HadGEM3-GC31-LL,0.08,0.01,0.14,0.05,0.11,0.09,0.08
INM-CM4-8,0.07,0.15,0.08,0.13,0.02,0.02,0.08


In [175]:
print(table_difference_cl1_styled_signi.to_latex(
    convert_css=True, hrules=True, position_float="centering",
    caption="Switching to Hypothesis' state-machines made.",
))

AttributeError: 'Styler' object has no attribute 'to_latex'

AttributeError: 'Styler' object has no attribute 'to_latex'